# So you want to start a podcast?

It seems that everybody has a podcast these days, and you want to get into the game. That is very understandable.

Podcast hosts can charge anywhere between [18 USD and 25 USD per every 1 000 listeners](https://www.podcastinsights.com/make-money-podcasting/) on sponsorships alone, in addition to setting up Patreon, paywalls, and other forms of monetizations. Given than [68 million people in the United States](https://www.podcastinsights.com/podcast-statistics/) alone listen to podcasts weekly, it is a lucrative business.

But what topic should you cover?

To answer that question, we'll be working with an iTunes database that contains over 46 000 podcasts and 1.2 million reviews. The reviews date between 2005 and 2021, which provides us with plenty of historical data. The full database can be [here](https://www.kaggle.com/thoughtvector/podcastreviews).

To determine the best topic and theme for a potential podcasts, we’ll do the following:

- Determine what categories have the highest rating
- Examine how that rating has changed over time
- Look at the most popular podcast within the most popular categories to uncover any trends 

If you anxious to know the answers right away, [you can jump straight to the results.](#5.-Final-thoughts) Otherise, let's analyze this data together.

In [1]:
!conda install -yc conda-forge ipython-sql

Solving environment: done

# All requested packages already installed.



In [2]:
%%capture
%load_ext sql
%sql sqlite:///database.sqlite

## 1. Data overview

Let's get a little overview of the data first.

<b>This database contains four tables - runs, podcasts, categories, and reviews.</b> The runs table is of no interest to us since it doesn't contain podcast related information. We'll be excluding it from our analysis. 

In [3]:
%%sql
SELECT type, name
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///database.sqlite
Done.


type,name
table,runs
table,podcasts
table,categories
table,reviews


Let's have a closer look at the remaining tables to see what information they contain. We'll being with Podcasts. 

It looks like each podcast has a unique ID as well as an iTunes ID. The slug and the title appear to be the same, only in slightly differnt format. In total, <b>we have 46 693 podcasts in this table.</b>

In [4]:
%%sql
SELECT *
FROM podcasts
LIMIT 5;

 * sqlite:///database.sqlite
Done.


podcast_id,itunes_id,slug,itunes_url,title
a00018b54eb342567c94dacfb2a3e504,1313466221,scaling-global,https://podcasts.apple.com/us/podcast/scaling-global/id1313466221,Scaling Global
a00043d34e734b09246d17dc5d56f63c,158973461,cornerstone-baptist-church-of-orlando,https://podcasts.apple.com/us/podcast/cornerstone-baptist-church-of-orlando/id158973461,Cornerstone Baptist Church of Orlando
a0004b1ef445af9dc84dad1e7821b1e3,139076942,mystery-dancing-in-the-dark,https://podcasts.apple.com/us/podcast/mystery-dancing-in-the-dark/id139076942,Mystery: Dancing in the Dark
a00071f9aaae9ac725c3a586701abf4d,1332508972,kts-money-matters,https://podcasts.apple.com/us/podcast/kts-money-matters/id1332508972,KTs Money Matters
a000aa69852b276565c4f5eb9cdd999b,1342447811,speedway-soccer,https://podcasts.apple.com/us/podcast/speedway-soccer/id1342447811,Speedway Soccer


In [5]:
%%sql
SELECT COUNT (DISTINCT podcast_id) tot_podcasts
FROM podcasts;

 * sqlite:///database.sqlite
Done.


tot_podcasts
46693


Category is next. The table is very simple, containing only the unique podcast ID and the corresponding category. <b> In total, there are 26 categories.</b> We’ll look at what they are later.

In [6]:
%%sql
SELECT *
FROM categories
LIMIT 1;

 * sqlite:///database.sqlite
Done.


podcast_id,category
c61aa81c9b929a66f0c1db6cbe5d8548,arts


In [7]:
%%sql

SELECT 
    COUNT(DISTINCT category) tot_categories
    FROM categories;

 * sqlite:///database.sqlite
Done.


tot_categories
26


Let's check if a given podcast belongs to one category or several:

In [8]:
%%sql
SELECT
    
    CASE
    WHEN count_cat = 1 THEN "1"
    WHEN count_cat = 2 THEN "2"
    WHEN count_cat = 3 THEN "3"
    WHEN count_cat = 4 THEN "4"
    WHEN count_cat = 5 THEN "5"
    WHEN count_cat = 6 THEN "6"
    WHEN count_cat = 7 THEN "7"
    END AS num_of_categories,
    
    COUNT(pod_id) num_of_pod,
    ROUND((CAST(COUNT(pod_id) AS FLOAT)/46693)*100, 2) pct
    
    FROM num_of_cat
    
    GROUP BY num_of_categories;

 * sqlite:///database.sqlite
Done.


num_of_categories,num_of_pod,pct
1,27853,59.65
2,13997,29.98
3,4259,9.12
4,556,1.19
5,25,0.05
6,3,0.01


We see that: 

- Almost 60% of all podcasts belong to only one category
- Roughly 30% have two categories
- The rest (10%) have three or more categories

Next, let's look at Review table:

In [9]:
%%sql
SELECT *
FROM reviews
LIMIT 1;

 * sqlite:///database.sqlite
Done.


podcast_id,title,content,rating,author_id,created_at
c61aa81c9b929a66f0c1db6cbe5d8548,really interesting!,Thanks for providing these insights. Really enjoy the variety and depth -- please keep them coming!,5,F7E5A318989779D,2018-04-24T12:05:16-07:00


Here, we have the full review, as well as rating and a unique author ID. 

In [10]:
%%sql
SELECT
    COUNT (*) tot_reviews
    FROM reviews;

 * sqlite:///database.sqlite
Done.


tot_reviews
1280337


#### Key observations

- The Podcasts table contains <b>46 693 podcasts</b> that spread across <b>26 categories</b>
- There are <b>1 280 337 reviews</b> in total
- <b>60% of all podcasts belong to only one category</b>, while the rest belong to two or more

## 2. An overview of podcast categories

Let's dive deeper into the 26 unique categories. First, we'll determine the number of podcasts in each one:

In [11]:
%%sql

SELECT 
    c.category,
    COUNT(*) tot_podcasts,
    ROUND(CAST(COUNT(*) AS FLOAT)/(
                                    SELECT 
                                    COUNT(*)
                                    FROM podcasts),2)*100 percentage
    FROM categories c 
    
    LEFT JOIN podcasts p ON p.podcast_id = c.podcast_id
    
    GROUP BY c.category
    ORDER BY tot_podcasts DESC;

 * sqlite:///database.sqlite
Done.


category,tot_podcasts,percentage
society-culture,10073,22.0
comedy,8080,17.0
religion-spirituality,7588,16.0
business,6208,13.0
arts,5576,12.0
education,5334,11.0
tv-film,5130,11.0
christianity,4997,11.0
music,4123,9.0
technology,2328,5.0


#### Key observations

- Some subject, such as art, business, and culture are split into subgroup, which makes sense since some topics can be very broad. Subgroups makes categorization more accurate.

- <b>Society-culture is the most saturated category</b> by far. It has over 10 000 different podcasts, which amounts to 22% of all podcasts on iTunes. 

- The two runnerups are very close in saturation - Comedy (17%) and Religion-Spirituality (16%).

- Out of the 26 categories, <b>the top three categories amount to 55% of all podcasts</b> on iTunes.


For further analysis, we'll focus on the ten categories with the most podcasts.

## 3. Podcast popularity

Next, let's focus on reviews and ratings. Sure, the Society-Culture category has the most podcasts. But are they any good?

By using the number of reviews and the overall rating, we can try to determine what categories are the most popular among listeners. 

Let's find the average rating for each top 10 category and the number of reviews they got.

In [12]:
%%sql

WITH top_10 AS 
          (SELECT category FROM (SELECT 
                c.category,
                COUNT(*) tot_podcasts

                FROM podcasts p

                LEFT JOIN categories c ON p.podcast_id = c.podcast_id

                GROUP BY c.category
                ORDER BY tot_podcasts DESC
                LIMIT 10
                                ))
                                


SELECT 
    c.category category,
    ROUND(AVG(r.rating),3) avg_rating,
    COUNT(*) num_of_reviews

    FROM podcasts p
    
    LEFT JOIN categories c ON p.podcast_id = c.podcast_id
    LEFT JOIN reviews r ON r.podcast_id = p.podcast_id 
    
    WHERE category IN top_10
    
    GROUP BY c.category
    ORDER BY 2 DESC;

 * sqlite:///database.sqlite
Done.


category,avg_rating,num_of_reviews
business,4.825,135458
religion-spirituality,4.811,95173
christianity,4.8,57903
education,4.795,119906
music,4.783,42617
arts,4.733,95862
comedy,4.634,256910
society-culture,4.541,286061
technology,4.536,35576
tv-film,4.518,124324


#### Key observations

- The average ratings don't actually vary significantly among the top 10 categories. The best category has an average rating of 4.825 while the worst has 4.518. Given that the top rating is 5, no category can be deemed bad.
- <b>The most popular category is business.</b> Religion-Spirituality and Christianity are in a very close second and third. 
- Christianity is, in a way, a subgenre of the Religion-Spirituality category, which reinforces <b>the popularity of the religious podcasts</b> even more.

### 3.1 Podcast popularity over time

Now that we know that Business, Religion-Spirituality, and Christianity are more popular than other categories, let's have a look the average rating over time.

Has popularity of different categories fluctuated a lot over the years?

To determine that, we need to first separate the data out into different time periods:

- three months
- six months
- one year
- five years
- all time

...and then calculate the average ratings for each period.

In [13]:
%%sql

DROP VIEW IF EXISTS three_m;
CREATE VIEW three_m AS
        SELECT 
            c.category category,
            ROUND(AVG(r.rating),3) avg_rating_3m

            FROM podcasts p

            LEFT JOIN categories c ON p.podcast_id = c.podcast_id
            LEFT JOIN reviews r ON r.podcast_id = p.podcast_id 

            WHERE (created_at LIKE "2021-10%" OR created_at LIKE "2021-09%" OR created_at LIKE "2021-08%")

            GROUP BY c.category;


DROP VIEW IF EXISTS six_m;            
CREATE VIEW six_m AS
        SELECT 
            c.category category,
            ROUND(AVG(r.rating),3) avg_rating_6m

            FROM podcasts p

            LEFT JOIN categories c ON p.podcast_id = c.podcast_id
            LEFT JOIN reviews r ON r.podcast_id = p.podcast_id 

            WHERE (created_at LIKE "2021-10%" 
                   OR created_at LIKE "2021-09%" 
                   OR created_at LIKE "2021-08%" 
                   OR created_at LIKE "2021-07%" 
                   OR created_at LIKE "2021-06%" 
                   OR created_at LIKE "2021-05%")

            GROUP BY c.category;


DROP VIEW IF EXISTS five_years;         
CREATE VIEW five_years AS
        SELECT 
            c.category category,
            ROUND(AVG(r.rating),3) avg_rating_5y

            FROM podcasts p

            LEFT JOIN categories c ON p.podcast_id = c.podcast_id
            LEFT JOIN reviews r ON r.podcast_id = p.podcast_id 

            WHERE (created_at LIKE "2021-%"
                   OR created_at LIKE "2020-%"
                   OR created_at LIKE "2019-%"
                   OR created_at LIKE "2018-%"
                   OR created_at LIKE "2017-%")

            GROUP BY c.category;
            
DROP VIEW IF EXISTS three_years;         
CREATE VIEW three_years AS
        SELECT 
            c.category category,
            ROUND(AVG(r.rating),3) avg_rating_3y

            FROM podcasts p

            LEFT JOIN categories c ON p.podcast_id = c.podcast_id
            LEFT JOIN reviews r ON r.podcast_id = p.podcast_id 

            WHERE (created_at LIKE "2021-%"
                   OR created_at LIKE "2020-%"
                   OR created_at LIKE "2019-%")

            GROUP BY c.category;
            
            
DROP VIEW IF EXISTS twelve_m; 
CREATE VIEW twelve_m AS
        SELECT 
            c.category category,
            ROUND(AVG(r.rating),3) avg_rating_12m

            FROM podcasts p

            LEFT JOIN categories c ON p.podcast_id = c.podcast_id
            LEFT JOIN reviews r ON r.podcast_id = p.podcast_id 

            WHERE (created_at LIKE "2021-10%" 
                   OR created_at LIKE "2021-09%" 
                   OR created_at LIKE "2021-08%" 
                   OR created_at LIKE "2021-07%" 
                   OR created_at LIKE "2021-06%" 
                   OR created_at LIKE "2021-05%"
                   OR created_at LIKE "2021-04%"
                   OR created_at LIKE "2021-03%"
                   OR created_at LIKE "2021-02%"
                   OR created_at LIKE "2021-01%"
                   OR created_at LIKE "2020-12%"
                   OR created_at LIKE "2020-11%")

            GROUP BY c.category;

            
DROP VIEW IF EXISTS all_ratings; 
CREATE VIEW all_ratings AS
SELECT 
    c.category category,
    ROUND(AVG(r.rating),3) avg_rating

    FROM podcasts p
    
    LEFT JOIN categories c ON p.podcast_id = c.podcast_id
    LEFT JOIN reviews r ON r.podcast_id = p.podcast_id 
    
    WHERE category IN top_10
    
    GROUP BY c.category
    ORDER BY 1 DESC;

 * sqlite:///database.sqlite
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [14]:
%%sql

WITH 
    top_10 AS 
          (SELECT category FROM (SELECT 
                c.category,
                COUNT(*) tot_podcasts

                FROM podcasts p

                LEFT JOIN categories c ON p.podcast_id = c.podcast_id

                GROUP BY c.category
                ORDER BY tot_podcasts DESC
                LIMIT 10
                                ))


SELECT 
    three_m.category,
    avg_rating_3m,
    avg_rating_6m,
    avg_rating_12m,
    avg_rating_3y,
    avg_rating_5y,
    avg_rating all_time
    
    FROM three_m
    
    INNER JOIN six_m ON three_m.category = six_m.category
    INNER JOIN twelve_m ON three_m.category = twelve_m.category
    INNER JOIN five_years ON three_m.category = five_years.category
    INNER JOIN all_ratings ar ON three_m.category = ar.category
    INNER JOIN three_years ON three_m.category = three_years.category
    
    WHERE three_m.category IN top_10
    
    GROUP BY three_m.category
    ORDER BY all_time DESC;

 * sqlite:///database.sqlite
Done.


category,avg_rating_3m,avg_rating_6m,avg_rating_12m,avg_rating_3y,avg_rating_5y,all_time
business,4.742,4.629,4.637,4.762,4.814,4.825
religion-spirituality,4.717,4.742,4.755,4.794,4.812,4.811
christianity,4.624,4.65,4.69,4.771,4.792,4.8
education,4.704,4.676,4.689,4.759,4.789,4.795
music,4.696,4.569,4.573,4.738,4.776,4.783
arts,4.571,4.521,4.543,4.689,4.73,4.733
comedy,4.386,4.351,4.421,4.545,4.606,4.634
society-culture,4.343,4.291,4.299,4.446,4.514,4.541
technology,3.53,3.568,3.775,4.305,4.46,4.536
tv-film,4.294,4.273,4.282,4.34,4.458,4.518


#### Key observations

- Apart from technology, no category has experienced a significant increase or decrease in popularity.
- <b>Business has seen a sligth dip and then an increase in popularity</b> if we look at the average ratings from three, six and twelve months ago. Those changes aren't very big.
- Podcasts related to <b>Regilion-Spirituality and Christianity have been sligthly less popular in the last years</b> but the change isn't significant either.

Based on this, we can conclude that <b> the popularity of Business, Religion-Spirituality, and Christianity podcasts holds up over time</b>. 

With that in mind, let's look closer at the three categories, and try to figure out what axactly within those broad topics interest people the most.

## 4. Top podcasts within business and religion

Let's look at the top 10 podcasts in each chosen category to see if the titles will shed any more light on what spesific topics interest people within those categories.

Here, we'll look at the number of reviews as one way to gauge engagement. Also, we'll use the all-time average rating as a benchmark to exclude any podcasts that get a lot of reviews but have a lower rating. 

In [15]:
%%sql

SELECT 
    p.title podcast_name,
    COUNT(*) num_of_reviews,
    ROUND(AVG(r.rating),3) avg_rating

    FROM podcasts p
    
    LEFT JOIN categories c ON p.podcast_id = c.podcast_id
    LEFT JOIN reviews r ON r.podcast_id = p.podcast_id 
    
    WHERE (c.category = "religion-spirituality") 
    
    GROUP BY 1
    HAVING avg_rating >=4.811
    ORDER BY 2 DESC
    LIMIT 10;

 * sqlite:///database.sqlite
Done.


podcast_name,num_of_reviews,avg_rating
Know Your Aura with Mystic Michaela,4339,4.988
Highest Self Podcast,1865,4.93
That Sounds Fun with Annie F. Downs,982,4.93
The Next Right Thing,830,4.943
Born to Impact,782,4.881
Journeywomen,760,4.975
Real Life Ghost Stories,720,4.957
Timothy Keller Sermons Podcast by Gospel in Life,709,4.882
Pints With Aquinas,649,4.908
The Positive Head Podcast,636,4.874


In [16]:
%%sql    
SELECT 
    p.title podcast_name,
    COUNT(*) num_of_reviews,
    ROUND(AVG(r.rating),3) avg_rating

    FROM podcasts p
    
    LEFT JOIN categories c ON p.podcast_id = c.podcast_id
    LEFT JOIN reviews r ON r.podcast_id = p.podcast_id 
    
    WHERE c.category = "christianity"
    
    GROUP BY 1
    HAVING avg_rating >=4.8
    ORDER BY 2 DESC
    LIMIT 10;

 * sqlite:///database.sqlite
Done.


podcast_name,num_of_reviews,avg_rating
That Sounds Fun with Annie F. Downs,982,4.93
The Next Right Thing,830,4.943
Born to Impact,782,4.881
Journeywomen,760,4.975
The Bible Binge,713,4.801
Timothy Keller Sermons Podcast by Gospel in Life,709,4.882
Pints With Aquinas,649,4.908
Exploring My Strange Bible,633,4.983
Heart of Dating,583,4.931
FILL IN THE BLANK with Carlos Whittaker,544,4.991


By looking at Religion-Spirituality and Christianity, we find:

- <b>Some podcasts appear in both categories</b>. For example, Journeywomen, That Sounds Fun with Annie F.Downs, and The Next Rigth Thing.
- The top podcasts within Religion-Spirituality seems to lean towards spiritual and even supernatural topics rather than spesific world religions. 
- Podcasts within <b>Religion-Spirituality category get more reviews</b> than Christianity podcasts, which can point towards a bigger audience.

In [17]:
%%sql    
SELECT 
    p.title podcast_name,
    COUNT(*) num_of_reviews,
    ROUND(AVG(r.rating),3) avg_rating

    FROM podcasts p
    
    LEFT JOIN categories c ON p.podcast_id = c.podcast_id
    LEFT JOIN reviews r ON r.podcast_id = p.podcast_id 
    
    WHERE c.category = "business"
    
    GROUP BY 1
    HAVING avg_rating >=4.825
    ORDER BY 2 DESC
    LIMIT 10;

 * sqlite:///database.sqlite
Done.


podcast_name,num_of_reviews,avg_rating
Don't Keep Your Day Job,2253,4.928
Jocko Podcast,1233,4.873
Sassy Lashes,1145,4.993
Macro Voices,839,4.862
Build Your Tribe | Grow Your Business with Social Media,679,4.963
The Playbook,627,4.861
Unstoppable Success Radio,615,4.985
Kwik Brain with Jim Kwik,607,4.896
Join Up Dots Podcast,592,4.985
The Smart Passive Income Online Business and Blogging Podcast,574,4.875


The Business category is more of a mixed of different topics. A bit of online research provides us with an overview of the spisific topics the top 10 podcasts cover:

- Don't Keep Your Day Job: caters to aspiring entreprenuers and creative freelancers
- Jocko Podcast: topics related to military
- Sassy Lashes: a podcast about the eye lash business
- Macro Voices: financial podcast covering the stock market
- Build Yout Tribe: social media marketing
- The Playbook: personal and professional advice from C-level professionals and enteprenuers
- Unstoppable Success Radio: personal and professional advice geared towards coaches, speakers and authors 
- Kwik Brain wtih Jim Kwik: personal growth 
- Join Up Dots Podcast: mindset and motivation for entreprenuers
- The Smart Passiv Income Online Business and Blogging: online business and creative business tips


A clear trend emerges among these top 10 podcasts: <b>enterprenuerial content seems to be very popular among listeners of Business podcasts.</b>

# 5. Final thoughts

So, what should your future podcast be about?

Based on the data you have three choices: Business, Religion-Spirituality, or Christianity. 

They are the highest rated categories of all times across 26 categories. They are also the only categories with an average rating above 4.8. 

- Business: 4.825
- Spirituality: 4.811
- Christianity: 4.8	

Their popularity hold up well over time. The lowest average rating each category has had is still pretty high:

- Business: 4.626
- Spirituality: 4.742
- Christianity: 4.624

### 5.1 Business
If you choose Business, your best bet is to target entreprenuers and provide some sort of professional advice. This is based on the top 10 podcasts in the category. 


### 5.2 Christianity and Religion-Spirituality
The topic of Christianity of pretty self-explanatory. 

When it comes to Religion-Spiritiality, however, you might want to focus on spiritual topics that do not pertain to any religion at all. As seen in the top 10 podcasts within Religion-Spirituality, a lot of them touch on subjects that either avoid religion, or have a christian undertone. There is also some overlap between top 10 Religion-Spirituality pocasts and top 10 Christianity podcasts. 

Religion-Spirituality podcasts might have the potential for a bigger audience compared to Christianity (judging by the number of reviews):

- Spirituality: 95173 total reviews
- Christianity: 57903 total reviews

But it's also the most saturated category of the two:

- Spirituality: 7588 podcasts	
- Christianity: 4997 podcasts

# 6. Bonus content: how many users listed to/rated more than one podcast

We already know that podcasts are a popular media to consume. But how many people listen to more than one podcast? 

To figure that out, we can look at the number of reviews they leave on each distinct podcast. Of cource, that still leaves the possibility that people listen to some podcasts than they review. But for the sake of this analysis, we'll put that aside.

In [18]:
%%sql    
DROP VIEW IF EXISTS pod_users; 
CREATE VIEW pod_users AS
    SELECT 
        r.author_id users,
        COUNT(DISTINCT p.title) num_of_podcasts,
        COUNT(*) num_of_reviews,

        CASE
        WHEN COUNT(DISTINCT p.title) = 1 THEN "1"
        WHEN COUNT(DISTINCT p.title) = 2 THEN "2"
        WHEN COUNT(DISTINCT p.title) = 3 THEN "3"
        WHEN COUNT(DISTINCT p.title) = 4 THEN "4"
        WHEN COUNT(DISTINCT p.title) = 5 THEN "5"
        WHEN COUNT(DISTINCT p.title) = 6 THEN "6"
        WHEN COUNT(DISTINCT p.title) = 7 THEN "7"
        WHEN COUNT(DISTINCT p.title) = 8 THEN "8"
        WHEN COUNT(DISTINCT p.title) = 9 THEN "9"
        WHEN COUNT(DISTINCT p.title) >= 10 THEN "over 10"
        END AS podcast_quantity

        FROM podcasts p

        LEFT JOIN categories c ON p.podcast_id = c.podcast_id
        LEFT JOIN reviews r ON r.podcast_id = p.podcast_id

        GROUP BY 1
        ORDER BY 2 DESC;

 * sqlite:///database.sqlite
Done.
Done.


[]

In [19]:
%%sql 
SELECT COUNT(DISTINCT users) FROM pod_users;

 * sqlite:///database.sqlite
Done.


COUNT(DISTINCT users)
755438


In [20]:
%%sql    
SELECT
    podcast_quantity,
    COUNT(*) num_of_users,
    ROUND((CAST(COUNT(*) AS FLOAT)/755438)*100, 3) user_pct
    
    FROM pod_users
    
    GROUP BY podcast_quantity 
    ORDER BY podcast_quantity ASC;

 * sqlite:///database.sqlite
Done.


podcast_quantity,num_of_users,user_pct
1,653768,86.542
2,69208,9.161
3,17639,2.335
4,6696,0.886
5,3012,0.399
6,1556,0.206
7,975,0.129
8,635,0.084
9,449,0.059
over 10,1500,0.199


We see that: 

- The majority of people, 86.5%, have rated only one podcast. 
- Almost 10% have rated two podcasts.
- 2.3% have rate three podcasts.
- Only 2% of people rate more than 4 podcasts.

This tells us that in <b>86% of the time, you only have one chance to capture the attention of the listener.</b>